In [ ]:
import geopandas as gpd

In [ ]:
map_wd = gpd.read_file('../world_map/Igismap/TM_WORLD_BORDERS-0.3.shp')

In [ ]:
map_wd.head()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig, ax = plt.subplots(1, figsize=(20,15))
fig = map_wd.plot(linewidth=0.2, edgecolor='black',ax=ax)

In [ ]:
sorted(map_wd.loc[:,'name'].to_list())

In [ ]:
map_cn = map_wd.loc[map_wd['name']=='China', :]
fig, ax = plt.subplots()
fig = map_cn.plot(linewidth=0.2, edgecolor='black', ax=ax)

In [ ]:
map_cn_tw = map_wd.loc[map_wd['name']=='Taiwan', :]
fig, ax = plt.subplots()
fig = map_cn_tw.plot(linewidth=0.2, edgecolor='black', ax=ax)

In [ ]:
map_cn.head()

In [ ]:
map_cn_tw.head()

In [ ]:
from shapely.geometry import MultiPolygon 
map_cn['geometry'].iloc[0] = MultiPolygon(list(map_cn['geometry'].iloc[0].geoms) + list(map_cn_tw['geometry'].iloc[0].geoms))

fig, ax = plt.subplots()
fig = map_cn.plot(linewidth=0.2, edgecolor='black', ax=ax)

In [ ]:
map_wd = map_wd.drop(map_wd.loc[map_wd['name']=='Taiwan', :].index)
map_wd.loc[map_wd['name']=='China',:] = map_cn

In [ ]:
fig, ax = plt.subplots(1, figsize=(20,15))
fig = map_wd.plot(linewidth=0.2, edgecolor='black',ax=ax)

In [ ]:
import pandas as pd

In [ ]:
# https://github.com/CSSEGISandData/COVID-19
covid_19_confirmed = pd.read_csv('../COVID-19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
covid_19_confirmed.loc[(covid_19_confirmed['Country/Region'] == 'Taiwan*'),'Province/State']='Taiwan'
covid_19_confirmed.loc[(covid_19_confirmed['Country/Region'] == 'Taiwan*'),'Country/Region']='China'

In [ ]:
covid_19_confirmed.head()

In [ ]:
list(covid_19_confirmed['Country/Region'])

In [ ]:
covid_19_countries = set()

for country in covid_19_confirmed['Country/Region']:
    covid_19_countries.add(country)

In [ ]:
len(covid_19_countries)

In [ ]:
covid_19_countries - set(map_wd.loc[:,'name'])

In [ ]:
rename_cvoid_19_countries = {
    'Brunei': 'Brunei Darussalam',
    'Cabo Verde': 'Capo Verde',
    'Congo (Brazzaville)': 'Congo',
    'Congo (Kinshasa)': 'Democratic Republic of the Congo',
    'Czechia': 'Czech Republic',
    'Eswatini': 'Swaziland',
    'Iran': 'Iran (Islamic Republic of)',
    'Korea, South': 'Korea, Republic of',
    'Laos': 'Lao People\'s Democratic Republic',
    'Libya': 'Libyan Arab Jamahiriya',
    'Moldova': 'Republic of Moldova',
    'North Macedonia': 'The former Yugoslav Republic of Macedonia',
    'Syria': 'Syrian Arab Republic',
    'Tanzania': 'United Republic of Tanzania',
    'US': 'United States',
    'Vietnam': 'Viet Nam'
}

In [ ]:
for k in rename_cvoid_19_countries:
    assert(covid_19_confirmed.loc[covid_19_confirmed['Country/Region']==k, 'Country/Region'].size == 1)
    
    covid_19_confirmed.loc[covid_19_confirmed['Country/Region']==k, 'Country/Region'] = rename_cvoid_19_countries[k]

In [ ]:
covid_19_countries = set()

for country in covid_19_confirmed['Country/Region']:
    covid_19_countries.add(country)

In [ ]:
covid_19_countries - set(map_wd.loc[:,'name'])

In [ ]:
covid_19_countries_in_map = list(covid_19_countries & set(map_wd.loc[:,'name']))

In [ ]:
len(covid_19_countries_in_map)

In [ ]:
sample_dates = covid_19_confirmed.columns[4:]
ds = dict()
for country in covid_19_countries_in_map:
    ds[country] = dict()
    for date in sample_dates:
        ds[country][date] = 0
        
for row in covid_19_confirmed.iterrows():
    name = row[1]['Country/Region']

    if name not in ds:
        print('\'{0}\' not in map'.format(name))
        continue
    
    for date in sample_dates:
        ds[name][date] = sum(covid_19_confirmed.loc[covid_19_confirmed['Country/Region']==name, date])
    

In [ ]:
from pandas import DataFrame

dd = dict()
dd['name'] = covid_19_countries_in_map

for date in sample_dates:
    dd[date] = list([0] * len(dd['name']))
    
    for i in range(len(dd['name'])):
        dd[date][i] = ds[dd['name'][i]][date]

covid_19_confirmed_in_map = DataFrame(data=dd)

In [ ]:
covid_19_confirmed_in_map.loc[covid_19_confirmed_in_map['name']=='China']

In [ ]:
from datetime import datetime
import matplotlib.colors as colors
import numpy as np

def plot_map(ds, output_fig=False):
    d = datetime.strptime(ds, '%Y-%m-%d')
    sample_date = d.strftime("%#m/%#d/%y")
    
    df = covid_19_confirmed_in_map.loc[:, ['name', sample_date]]
    
    map_with_covid_19 = map_wd.set_index('name').join(df.set_index('name'))
    
    vmin = 0
    vmax = 1e+7
    
    fig, ax = plt.subplots(1, figsize=(20,15))
    ax.axis('off')
    ax.set_title(ds)
    
    #if use_lognorm:
    #    fig = map_with_covid_19.plot(column=sample_date, cmap='Reds', linewidth=0.2, edgecolor='green', figsize=(30,30), legend=True, ax=ax, norm=colors.LogNorm(vmin=vmin, vmax=vmax), vmin=vmin, vmax=vmax)
    #else:
    #    fig = map_with_covid_19.plot(column=sample_date, cmap='Reds', linewidth=0.2, edgecolor='green', figsize=(30,30), legend=True, ax=ax, norm=plt.Normalize(vmin=vmin, vmax=vmax), vmin=vmin, vmax=vmax)
    
    
    bounds = np.array([0, 1, 1e+1, 1e+2, 1e+3, 1e+4, 1e+5, 1e+6, 1e+7])
    norm = colors.BoundaryNorm(boundaries=bounds, ncolors=256)
    
    fig = map_with_covid_19.plot(column=sample_date, cmap='Reds', linewidth=0.2, edgecolor='black', legend=True, ax=ax, norm=norm, vmin=vmin, vmax=vmax)

    #ax.legend(shadow=True, fancybox=True)
    
    if output_fig:
        plt.savefig('c:/tmp/fig/World_{0}.png'.format(ds))
  

In [ ]:
plot_map('2020-1-22', True)

In [ ]:
plot_map('2020-2-22', True)

In [ ]:
plot_map('2020-3-22', True)

In [ ]:
plot_map('2020-4-22', True)

In [ ]:
plot_map('2020-5-22', True)

In [ ]:
plot_map('2020-6-22', True)

In [ ]:
plot_map('2020-7-22', True)

In [ ]:
plot_map('2020-8-22', True)

In [ ]:
plot_map('2020-9-22', True)